## Combine TT DF with Demographic Info and Calculate Index
Author: Callie Clark
Last Updated: 6/8/2024

In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import networkx as nx

%matplotlib inline
from TT_functions import *
import random
import json

In [67]:
def aggregate_tt(df_tt, time_period_colnames,cutoff):
    df_tt['min']=df_tt.loc[:,time_period_colnames].min(axis=1)
    df_tt['mean']=df_tt.loc[:,time_period_colnames].mean(axis=1)

    df_tt['index']=(((df_tt.loc[:,time_period_colnames]<cutoff)*1).sum(axis=1))/len(time_period_colnames)
   
    
    print('# periods',len(time_period_colnames))
    return df_tt[(time_period_colnames+['min','mean','index'])]



### Generate DF
* Combine Mode Travel Times with demographics
* calculate index and poverty indicator?

In [75]:
df_demo_=pd.read_csv('CT_demog_FI_TCL.csv',index_col=0)

csv_list=['df_tt_S1.csv', 
          'df_tt_S2.csv',
 'df_tt_drive_k10.csv',
 'df_tt_transit_k25.csv',
 'df_tt_bike_k10.csv',
 'df_tt_walk_k10.csv']

In [76]:
for csv in csv_list:
    df_tt_=pd.read_csv('travel_time_df/'+csv,index_col=0)
    df_tt_.columns=df_tt_.columns.str.replace('_test','')
    df_tt_.columns=df_tt_.columns.str.replace('_tt_25','')
    tt_3_colnames=[i for i in df_tt_.columns if 'tt_' in i]
    tt_1_colnames=[i for i in df_tt_.columns if 'tt' in i and 'tt_3' not in i]
    
    df_tt_agg_1=aggregate_tt(df_tt_, tt_1_colnames,cutoff=20)
    df_tt_1=df_tt_agg_1.merge(df_demo_,how='right',right_index=True,left_index=True).drop_duplicates()
    #cite:https://data.census.gov/table/ACSDT5Y2021.B19019?q=median%20income%20household%20size&g=310XX00US35620
    med_hh_inc_msa=86445
    df_tt_1['poverty_ind']=((df_tt_1['percent_poverty']>=0.2)|(df_tt_1['Median_Income']<=(med_hh_inc_msa*0.8)))*1 #add poverty indicator
    df_tt_1.to_csv('processed_df/'+csv[:-4]+'.csv')
    
    df_tt_agg_3=aggregate_tt(df_tt_, tt_3_colnames,cutoff=20)
    df_tt_3=df_tt_agg_3.merge(df_demo_,how='right',right_index=True,left_index=True).drop_duplicates()
    df_tt_3['poverty_ind']=((df_tt_3['percent_poverty']>=0.2)|(df_tt_3['Median_Income']<=(med_hh_inc_msa*0.8)))*1 #add poverty indicator
    df_tt_3.to_csv('processed_df/'+csv[:-4]+'_3_.csv')

# periods 1
# periods 1
# periods 28
# periods 28
# periods 28
# periods 28
# periods 28
# periods 28
# periods 28
# periods 28
# periods 28
# periods 28


## Create Combined Df by Mode

In [77]:
# data on mode based on vehicle use from https://www.nyc.gov/html/dot/html/about/citywide-mobility-survey.shtml
no_vehicles={'walk':round(0.53/0.92,4), 'drive':round(0.06/0.92,4), 'transit':round(0.33/0.92,4)}
vehicles={'walk':round(0.30/0.93,4), 'drive':round(0.49/0.93,4), 'transit':round(0.14/0.93,4)}
#create dataframe where the mode is weighted by frequency wrt car ownership
mode_by_veh_owner=df_demo_[['percent_car_owners']]
mode_by_veh_owner['walk_weighted']=round(mode_by_veh_owner.percent_car_owners*vehicles['walk']+(1-mode_by_veh_owner.percent_car_owners)*no_vehicles['walk'],4)
mode_by_veh_owner['drive_weighted']=round(mode_by_veh_owner.percent_car_owners*vehicles['drive']+(1-mode_by_veh_owner.percent_car_owners)*no_vehicles['drive'],4)
mode_by_veh_owner['transit_weighted']=round(mode_by_veh_owner.percent_car_owners*vehicles['transit']+(1-mode_by_veh_owner.percent_car_owners)*no_vehicles['transit'],4)
mode_by_veh_owner.head()

/tmp/ipykernel_2539/4027503607.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mode_by_veh_owner['walk_weighted']=round(mode_by_veh_owner.percent_car_owners*vehicles['walk']+(1-mode_by_veh_owner.percent_car_owners)*no_vehicles['walk'],4)
/tmp/ipykernel_2539/4027503607.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mode_by_veh_owner['drive_weighted']=round(mode_by_veh_owner.percent_car_owners*vehicles['drive']+(1-mode_by_veh_owner.percent_car_owners)*no_vehicles['drive'],4)
/tmp/ipykernel_2539/40275

,percent_car_owners,walk_weighted,drive_weighted,transit_weighted
GEOID,,,,
36005000200,0.7644,0.3823,0.4181,0.1996
36005000400,0.7449,0.3873,0.4091,0.2036
36005001600,0.4449,0.4633,0.2706,0.2661
36005001901,0.3051,0.4988,0.2061,0.2952
36005001902,0.3564,0.4858,0.2297,0.2845


In [78]:
display(no_vehicles)
display(vehicles)


{'walk': 0.5761, 'drive': 0.0652, 'transit': 0.3587}

{'walk': 0.3226, 'drive': 0.5269, 'transit': 0.1505}

In [79]:
mode_by_veh_owner.describe()

,percent_car_owners,walk_weighted,drive_weighted,transit_weighted
count,2196.000000,2196.000000,2196.000000,2196.000000
mean,0.516439,0.445184,0.303640,0.251177
std,0.242483,0.061468,0.111954,0.050484
min,0.029300,0.322600,0.078700,0.150500
25%,0.311425,0.392175,0.208975,0.207675
50%,0.496350,0.450300,0.294350,0.255350
75%,0.725450,0.497125,0.400150,0.293825
max,1.000000,0.568700,0.526900,0.352600


In [80]:
#create combined df
drive_df=pd.read_csv('processed_df/df_tt_drive_k10_3.csv',index_col=0)
transit_df=pd.read_csv('processed_df/df_tt_transit_k25_3.csv',index_col=0)
walk_df=pd.read_csv('processed_df/df_tt_walk_k10_3.csv',index_col=0)

#use df with average tt to three closest pantries
tt_3_colnames=[i for i in transit_df.columns if 'tt_' in i]
df_transit_weighted=transit_df.loc[:,tt_3_colnames].multiply(mode_by_veh_owner['transit_weighted'], axis="index")
df_walk_weighted=walk_df.loc[:,tt_3_colnames].multiply(mode_by_veh_owner['walk_weighted'], axis="index")
df_drive_weighted=drive_df.loc[:,tt_3_colnames].multiply(mode_by_veh_owner['drive_weighted'], axis="index")
df_comb_tt=df_transit_weighted+df_walk_weighted+df_drive_weighted
df_comb_tt_=aggregate_tt(df_comb_tt.round(2), tt_3_colnames,cutoff=20)
df_comb_tt_final=df_comb_tt_.merge(df_demo_,how='right',right_index=True,left_index=True).drop_duplicates()
df_comb_tt_final['poverty_ind']=((df_comb_tt_final['percent_poverty']>=0.2)|(df_comb_tt_final['Median_Income']<=(med_hh_inc_msa*0.8)))*1 #add poverty indicator
df_comb_tt_final.to_csv('processed_df/df_tt_EFAI_3.csv')

# periods 28


In [81]:
#create combined df
drive_df=pd.read_csv('processed_df/df_tt_drive_k10.csv',index_col=0)
transit_df=pd.read_csv('processed_df/df_tt_transit_k25.csv',index_col=0)
walk_df=pd.read_csv('processed_df/df_tt_walk_k10.csv',index_col=0)

#use df with tt to closest pantries
tt_1_colnames=[i for i in df_tt_.columns if 'tt' in i and 'tt_3' not in i]
df_transit_weighted=transit_df.loc[:,tt_1_colnames].multiply(mode_by_veh_owner['transit_weighted'], axis="index")
df_walk_weighted=walk_df.loc[:,tt_1_colnames].multiply(mode_by_veh_owner['walk_weighted'], axis="index")
df_drive_weighted=drive_df.loc[:,tt_1_colnames].multiply(mode_by_veh_owner['drive_weighted'], axis="index")
df_comb_tt=df_transit_weighted+df_walk_weighted+df_drive_weighted
df_comb_tt_=aggregate_tt(df_comb_tt.round(2), tt_1_colnames,cutoff=20)
df_comb_tt_final=df_comb_tt_.merge(df_demo_,how='right',right_index=True,left_index=True).drop_duplicates()
df_comb_tt_final['poverty_ind']=((df_comb_tt_final['percent_poverty']>=0.2)|(df_comb_tt_final['Median_Income']<=(med_hh_inc_msa*0.8)))*1 #add poverty indicator
df_comb_tt_final.to_csv('processed_df/df_tt_EFAI.csv')

# periods 28
